[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel/e2eAIOK/blob/main/demo/builtin/dlrm/DLRM_DEMO.ipynb)

# DLRM DEMO

Deep Learning Recommendation Model for Personalization and Recommendation Systems

* original source
    * Source repo: https://github.com/facebookresearch/dlrm


# Content
* [Overview](#Overview)
    * [Model Architecture](#Model-Architecture)
    * [Optimizations](#Optimizations)
    * [Performance](#Performance)
* [Getting Started](#Getting-Started)
    * [1. Environment Setup](#1.-Environment-Setup)
    * [2. Workflow Prepare](#2.-Workflow-Prepare)
    * [3. Data Prepare](#3.-Data-Prepare)
    * [4. Train](#4.-Train)

------

# Overview

## Model Architecture

<img src="./img/dlrm.png" alt="DLRM network" style="width: 400px;"/>


## Optimizations

### New Imports
``` diff
+ import dlrm_data_pytorch as dp
+ from lamb_bin import Lamb, log_lamb_rs
+ # For distributed run
+ import extend_distributed as ext_dist
+ try:
+     import intel_pytorch_extension as ipex
+     from intel_pytorch_extension import core
+ except:
+     pass

- import torch.nn.functional as F
- import extend_distributed as ext_dist
```

### MLP layer democratization

``` diff
def create_mlp(self, ln, sigmoid_layer):
    # build MLP layer by layer
    layers = nn.ModuleList()
    for i in range(0, ln.size - 1):
        n = ln[i]
        m = ln[i + 1]

        # construct fully connected operator
+           LL = ipex.IpexMLPLinear(int(n), int(m), bias=True, output_stays_blocked=(i < ln.size - 2), default_blocking=32)
-           LL = nn.Linear(int(n), int(m), bias=True)

        ...
+           LL.to(torch.bfloat16)
+           if hasattr(LL, 'reset_weight_shape'):
+               LL.reset_weight_shape(block_for_dtype=torch.bfloat16)
        ...
```

### Embedding layer democratization

``` diff
def create_emb(self, m, ln, local_ln_emb_sparse=None, ln_emb_dense=None):
    emb_l = nn.ModuleList()
+   # save the numpy random state
+   np_rand_state = np.random.get_state()
     ...
    for i in embs:
         ...
        # construct embedding operator, original all embedding are in same dimension            
-       m_curr = (m[i] if self.max_emb_dim > 0 else m)
-       EE = nn.EmbeddingBag(n, m_curr, mode="sum", sparse=True)
-       W = np.random.uniform(
-           low=-np.sqrt(1 / n), high=np.sqrt(1 / n), size=(n, m_curr)
-       ).astype(np.float32)

        # democratized, use two dimension, sparse and dense
+       W = np.random.uniform(
+           low=-np.sqrt(1 / n), high=np.sqrt(1 / n), size=(n, m)
+       ).astype(np.float32)
+       if n >= self.sparse_dense_boundary:
+           m_sparse = int(m/4)
+           W = np.random.uniform(
+               low=-np.sqrt(1 / n), high=np.sqrt(1 / n), size=(n, m_sparse)
+           ).astype(np.float32)
+           EE = nn.EmbeddingBag(n, m_sparse, mode="sum", sparse=True, _weight=torch.tensor(W, requires_grad=True))
+       else:
+           W = np.random.uniform(
+               low=-np.sqrt(1 / n), high=np.sqrt(1 / n), size=(n, m)
+           ).astype(np.float32)
+           EE = nn.EmbeddingBag(n, m, mode="sum", sparse=False, _weight=torch.tensor(W, requires_grad=True))
+           tensor(W, requires_grad=True))

        # cast to BF16
+       EE.to(torch.bfloat16)
        ...
```

### Adding distributed training in forward function

``` diff
def forward(self, dense_x, lS_o, lS_i):
+   if self.bf16:
+       dense_x = dense_x.bfloat16()
+   if ext_dist.my_size > 1:
+       return self.distributed_forward(dense_x, lS_o, lS_i)
    if self.ndevices <= 1:
        return self.sequential_forward(dense_x, lS_o, lS_i)
    else:
        return self.parallel_forward(dense_x, lS_o, lS_i)
```

### Expanding Optimizer Options

``` diff
- optimizer = torch.optim.SGD(dlrm.parameters(), lr=args.learning_rate)
+ optimizer_list = ([torch.optim.SGD, ([Lamb, False], torch.optim.SGD),
+                    torch.optim.Adagrad, ([torch.optim.Adam, None], torch.optim.SparseAdam)],
+                   [ipex.SplitSGD, ([Lamb, True], ipex.SplitSGD)])
+ optimizers = optimizer_list[args.bf16 and ipex.is_available()][args.optimizer]
+ optimizer_dense = optimizers[0][0]([
+     {"params": [p for emb in dlrm.emb_dense for p in emb.parameters()], "lr": args.lamblr},
+     {"params": dlrm.bot_l.parameters(), "lr": args.lamblr},
+     {"params": dlrm.top_l.parameters(), "lr": args.lamblr}
+ ], lr=args.lamblr, bf16=args.bf16)
+ optimizer_sparse = optimizers[1]([
+     {"params": [p for emb in dlrm.emb_sparse for p in emb.parameters()],
+      "lr": args.learning_rate / ext_dist.my_size},
+ ], lr=args.learning_rate)
+ optimizer = (optimizer_dense, optimizer_sparse)
```

### HPO with SDA (Smart Democratization Advisor)
SDA config

Parameters for SDA auto optimization:
- learning_rate: 5 ~ 50
- lamb_lr: 5 ~ 50
- warmup_steps: 2000 ~ 4500
- decay_start_steps: 4501 ~ 9000
- num_decay_steps: 5000 ~ 15000
- sparse_feature_size: [128, 64, 16]
- mlp_top_size: ["1024-1024-512-256-1","512-512-256-128-1","512-256-128-1","512-256-1","256-128-1","128-64-1","256-1","128-1"]
- mlp_bot_size = ["13-512-256-","13-512-256-","13-256-","13-128-"]
            

metrics:
- name: accuracy
  objective: maximize
- name: training_time
  strategy: optimize
  objective: minimize
  

## Performance


![dlrm_performance](./img/dlrm_perf.png)

### Key Optimizations

* Intel Optimized training framework + Lamb optimizer: 1.29x speedup (1.64 to 2.13)​

* LAMB + BF16: 1.45x speedup (2.87 to 4.18)​

* Lighter Model-part1: model optimization(decrease both dense embedding and sparse embedding output dimension from 128 to 64, reduced bot_mlp layer size from 13-512-256-128 to 13-256-128-64, top_mlp layer size from 1024-1024-512-256-1 to 512-512-256-1) delivers 2.17x speedup(4.18 to 9.07)​

* Optimizer: lamb optimizer delivers 1.04x speedup(9.07 to 9.50)​

* Lighter Model-part2: model optimization(reduced bot_mlp layer size from 13-256-128-64 to 13-128-64, top_mlp layer size from 512-512-256-1 to 256-128-1) delivers 1.22x speedup(9.50 to 11.61) ​

* Embedding: reducing sparse embedding table number from 16 to 8 delivers 1.36x speedup(11.61 to 15.85) ​

* Framework optimization: using latest IPEX launch scripts (auto bind CCL to specific core) and add KMP setting delivers 1.06x speedup (15.85 to 16.90)​

* AlltoAll optimization: reduce the output dimension size from 64 to 16 and then repeat 4 times to 64 dimension after alltoall, which delivers 1.25x speedup(16.90 to 21.24)​

* Learning rate and test optimization: change lamb optimizer learning rate from 16 to 30;remove gradient computing in the test part; increase test batch size from 16K to 128K. These optimizations delivers 1.30x speedup(21.24 to 27.71)​

* Other optimization: (1) Using NVMe to replace HDD as storage;(2) Leveraging prefetch_generator PyTorch dataloader optimization package to enable batch data generator working in background thread parallelly with training. These optimizations delivers 1.08x speedup(27.71 to 29.93) ​

* SigOpt optimization: In limit experiments trials, SigOpt with metrics threshold delivers 1.06x speedup(29.93 to 31.89) than previous manual optimized result.​

------


# Getting Started
Noted: This demo needs to compile torch v1.5.0-rc3, ipex, torchCCL package, it was tested on bare metal environment. Due to the limitations of colab system, this notebook is only for presentation purpose.

  * [1. Environment Setup](#1.-Environment-Setup)
  * [2. Workflow Prepare](#2.-Workflow-Prepare)
  * [3. Data Prepare](#3.-Data-Prepare)
  * [4. Train](#4.-Train)
  * [5. Inference](#5.-Inference)

## 1. Enviroment Setup

(Option 1) use pip - DLRM model is based on DLRM MLPerf Training 0.7 Intel submission

  * source: https://github.com/mlperf/training_results_v0.7/tree/master/Intel/benchmarks/dlrm/1-node-4s-cpx-pytorch
  * It has strict dependencies to GCC version, torch version, oneCCL version.

In [ ]:
# install GCC and dependencies
! conda install gxx_linux-64==8.4.0
! pip install sklearn onnx tqdm lark-parser
! pip install -e git+https://github.com/mlperf/logging@0.7.0-rc2#egg=logging
! conda config --append channels intel
! conda install ninja pyyaml setuptools cmake cffi typing
! conda install intel-openmp mkl mkl-include numpy -c intel --no-update-deps
! conda install -c conda-forge gperftools
# git clone pytorch v1.5.0-rc3
! git clone https://github.com/pytorch/pytorch.git && cd pytorch && git checkout tags/v1.5.0-rc3 -b v1.5-rc3 && git submodule sync && git submodule update --init --recursive
# git clone ipex v0.2
! git clone https://github.com/intel/intel-extension-for-pytorch.git && cd intel-extension-for-pytorch && git checkout tags/v0.2 -b v0.2 && git submodule sync && git submodule update --init --recursive
# apply ipex patch to pytorch and install
! cd intel-extension-for-pytorch && cp torch_patches/0001-enable-Intel-Extension-for-CPU-enable-CCL-backend.patch ../pytorch/ && cd ../pytorch && patch -p1 < 0001-enable-Intel-Extension-for-CPU-enable-CCL-backend.patch
! cd pytorch && python setup.py install
! cd intel-extension-for-pytorch && python setup.py install
# git clone oneCCL and install
! git clone https://github.com/oneapi-src/oneCCL.git && cd oneCCL && git checkout 2021.1-beta07-1 && mkdir build && cd build && cmake .. -DCMAKE_INSTALL_PREFIX=~/.local && make install -j
# git clone torchCCL and install
! git clone https://github.com/intel/torch-ccl.git && cd torch-ccl && git checkout 2021.1-beta07-1
! source ~/.local/env/setvars.sh && cd torch-ccl && python setup.py install
! python -m pip install onnx tqdm lark-parser pyyaml prefetch_generator tensorboardX psutil sigopt pandas pyarrow lightgbm transformers xgboost

# install pyrecdp and e2eAIOK-sda
! python -m pip install pyrecdp
! python -m pip install e2eAIOK-sda --pre

(Option 2) use docker

``` bash
# 1. git clone codes
git clone https://github.com/intel/e2eAIOK.git
cd e2eAIOK
git submodule update --init --recursive

# 2. build docker image
python3 scripts/start_e2eaiok_docker.py -b pytorch -w ${host0} ${host1} ${host2} ${host3} --proxy "http://addr:ip"

# 3. Enter Docker
sshpass -p docker ssh ${host0} -p 12344

# 4. start jupyter notebook
nohup jupyter notebook --notebook-dir=/home/vmagent/app/e2eaiok --ip=${hostname} --port=8899 --allow-root &
Now you can visit demso in http://${hostname}:8899/.
```

## 2. Workflow Prepare

In [ ]:
! sh workflow_prepare_dlrm.sh

## 3. Data Prepare

Download from https://labs.criteo.com/2013/12/download-terabyte-click-logs/ and unzip them
``` bash
ls criteo/raw_data
day_0  day_10  day_12  day_14  day_16  day_18  day_2   day_21  day_23  day_4  day_6  day_8
day_1  day_11  day_13  day_15  day_17  day_19  day_20  day_22  day_3   day_5  day_7  day_9
```

lauch data process to downloaded files.
original data contains 23 large csv files, below script will do Data Conversion from CSV to parquet files.

In [16]:
! cd e2eaiok/modelzoo/dlrm/data_processing/; python convert_to_parquet.py --dataset_path criteo

sr140
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/29 05:51:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/29 05:51:25 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
per core memory size is 7.500 GB and shuffle_disk maximum capacity is 800.000 GB
save data to file:////home/vmagent/app/dataset/criteo_small/output//dlrm_parquet_train_day_0
Convert day_0 to parquet completed, took 104.0467173489742 secs                 
Splitting the last day into 2 parts of test and validation...
head: cannot open '/home/vmagent/app/dataset/criteo_small/raw_data//day_23' for reading: No such file or directory
tail: cannot

do data process to convert all 23 files to ready-to-train parquet files.

In [7]:
! cd e2eaiok/modelzoo/dlrm/data_processing/; python preprocessing.py --dataset_path criteo

sr140
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/29 04:18:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/29 04:18:04 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
recdp-scala-extension is enabled
per core memory size is 7.500 GB and shuffle_disk maximum capacity is 800.000 GB
save data to file:////home/vmagent/app/dataset/criteo_small/output//dlrm_parquet_train_proc_day_0
Process day_0 categorified columns completed, took 317.1812623520382 secs       
save data to file:////home/vmagent/app/dataset/criteo_small/output//dlrm_parquet_test_proc
Process test categorified columns completed, took 1.60864770

DLRM model need to consume numpy files, do data conversion from parquet to numpy binary files.

In [15]:
! cd e2eaiok/modelzoo/dlrm/data_processing/; python convert_to_binary.py --dataset_path criteo

Start to generate day_fea_count.npz
Get dimension of _c14
Get dimension of _c15
Get dimension of _c16
Get dimension of _c17
Get dimension of _c18
Get dimension of _c19
Get dimension of _c20
Get dimension of _c21
Get dimension of _c22
Get dimension of _c23
Get dimension of _c24
Get dimension of _c25
Get dimension of _c26
Get dimension of _c27
Get dimension of _c28
Get dimension of _c29
Get dimension of _c30
Get dimension of _c31
Get dimension of _c32
Get dimension of _c33
Get dimension of _c34
Get dimension of _c35
Get dimension of _c36
Get dimension of _c37
Get dimension of _c38
Get dimension of _c39
[14885288, 29419, 15123, 7291, 19899, 3, 6463, 1310, 61, 10155909, 618195, 218994, 10, 2208, 9779, 71, 4, 963, 14, 16967044, 4154705, 13180313, 289595, 10828, 95, 34]
Start to convert train/valid/test
multi process is 6
Create subprocess 0 for train_data.bin[0:1], total 1
Start to convert parquet files to numpy binary
Start to write binary to /home/vmagent/app/dataset/criteo_small/output/t

# 4. Train

SDA is an autoHPO component, we use SDA to trigger DLRM training and validation.
Noticed: set enable_sigopt to True, SDA will explore HyperParameter for this model. In our demo, we will use our searched best parameter.

In [1]:
from e2eAIOK.SDA.SDA import SDA

settings = dict()
settings["data_path"] = "criteo/"
settings["ppn"] = 2
settings["ccl_worker_num"] = 4
settings["enable_sigopt"] = False
settings["train_script"] = "e2eaiok/modelzoo/dlrm/dlrm/"

sda = SDA(model="dlrm", settings=settings) # default settings
sda.launch()

hydro_model = sda.snapshot()
hydro_model.explain()

data format is binary

***    Best Trained Model    ***
  Model Type: dlrm
  Model Saved Path: 
  Sigopt Experiment id is None
  === Result Metrics ===
[1] torch_ccl is True
[0] torch_ccl is True
[1] :::MLLOG {"namespace": "", "time_ms": 1666937085214, "event_type": "POINT_IN_TIME", "key": "cache_clear", "value": true, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/dlrm/dlrm/dlrm_s_pytorch.py", "lineno": 646}}
[1] :::MLLOG {"namespace": "", "time_ms": 1666937085294, "event_type": "INTERVAL_START", "key": "init_start", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/dlrm/dlrm/dlrm_s_pytorch.py", "lineno": 648}}
[1] world_size:2,rank:1
[0] :::MLLOG {"namespace": "", "time_ms": 1666937085224, "event_type": "POINT_IN_TIME", "key": "cache_clear", "value": true, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/dlrm/dlrm/dlrm_s_pytorch.py", "lineno": 646}}
[0] :::MLLOG {"namespace": "", "time_ms": 1666937085298, "event_type": "INTERVAL_START", "key": 

[1] Model created!
[1] DLRM_Net(
[1]   (emb_dense): ModuleList(
[1]     (0): EmbeddingBag(39043, 128, mode=sum)
[1]     (1): EmbeddingBag(17289, 128, mode=sum)
[1]     (2): EmbeddingBag(7420, 128, mode=sum)
[1]     (3): EmbeddingBag(20263, 128, mode=sum)
[1]     (4): EmbeddingBag(3, 128, mode=sum)
[1]     (5): EmbeddingBag(7120, 128, mode=sum)
[1]     (6): EmbeddingBag(1543, 128, mode=sum)
[1]     (7): EmbeddingBag(63, 128, mode=sum)
[1]     (8): EmbeddingBag(10, 128, mode=sum)
[1]     (9): EmbeddingBag(2208, 128, mode=sum)
[1]     (10): EmbeddingBag(11938, 128, mode=sum)
[1]     (11): EmbeddingBag(155, 128, mode=sum)
[1]     (12): EmbeddingBag(4, 128, mode=sum)
[1]     (13): EmbeddingBag(976, 128, mode=sum)
[1]     (14): EmbeddingBag(14, 128, mode=sum)
[1]     (15): EmbeddingBag(12972, 128, mode=sum)
[1]     (16): EmbeddingBag(108, 128, mode=sum)
[1]     (17): EmbeddingBag(36, 128, mode=sum)
[1]   )
[1]   (emb_sparse): ModuleList(
[1]     (0): EmbeddingBag(39979771, 32, mode=sum)
[1] 

[0] Finished training it 176/256 of epoch 0, 5016.83 ms/it, loss 0.129738, accuracy 96.677 %
[1] Finished training it 176/256 of epoch 0, 5018.03 ms/it, loss 0.129163, accuracy 96.691 %
[0] Finished training it 192/256 of epoch 0, 5024.25 ms/it, loss 0.128225, accuracy 96.722 %
[1] Finished training it 192/256 of epoch 0, 5024.32 ms/it, loss 0.128767, accuracy 96.701 %
[1] Finished training it 208/256 of epoch 0, 5018.23 ms/it, loss 0.128546, accuracy 96.701 %
[0] Finished training it 208/256 of epoch 0, 5019.18 ms/it, loss 0.128171, accuracy 96.715 %
[0] Finished training it 224/256 of epoch 0, 5022.23 ms/it, loss 0.128718, accuracy 96.699 %
[1] Finished training it 224/256 of epoch 0, 5022.90 ms/it, loss 0.128312, accuracy 96.708 %
[0] Finished training it 240/256 of epoch 0, 5010.63 ms/it, loss 0.128252, accuracy 96.703 %
[1] Finished training it 240/256 of epoch 0, 5010.63 ms/it, loss 0.128856, accuracy 96.694 %
[0] Finished training it 256/256 of epoch 0, 5014.20 ms/it, loss 0.128

# 5. Inference

In [14]:
! cd e2eaiok/modelzoo/dlrm/; bash run_inference.sh

/opt/intel/oneapi/intelpython/latest/envs/pytorch_mlperf/bin/python -u ./dlrm/launch.py --distributed --nproc_per_node=2 ./dlrm/dlrm_s_pytorch_inference.py --eval-data-path=/home/vmagent/app/dataset/criteo/test_data.bin --day-feature-count=/home/vmagent/app/dataset/criteo/day_fea_count.npz --load-model=./result/ --arch-sparse-feature-size=64 --arch-mlp-bot=13-128-64 --arch-mlp-top=256-128-1 --max-ind-range=40000000 --data-generation=dataset --data-set=terabyte --raw-data-file=/home/vmagent/app/dataset/criteo/day --processed-data-file=/home/vmagent/app/dataset/criteo/terabyte_processed.npz --loss-function=bce --round-targets=True --bf16 --num-workers=0 --test-num-workers=0 --use-ipex --optimizer=1 --dist-backend=ccl --learning-rate=16 --mini-batch-size=262144 --print-freq=16 --print-time --test-freq=800 --sparse-dense-boundary=403346 --test-mini-batch-size=131072 --lamblr=30 --lr-num-warmup-steps=4000 --lr-decay-start-step=5760 --lr-num-decay-steps=27000 --memory-map --mlperf-logging --